# Лабораторная работа №4
## Хеширование

### Задание №1
Возьмите реализацию класса HashTable из лекционных материалов и выполните
следующие доработки:

1. Реализуйте квадратичное пробирование как технику повторного хеширования.

2. Реализуйте работу с функцией len (переопределите метод __len__).

3. Реализуйте работу оператора in (переопределите метод __contains__).

4. Переделайте метод put таким образом, чтобы таблица автоматически меняла размер,
когда загрузочный фактор становится больше значения 0.7. Размер должен
увеличиваться примерно в два раза до ближайшего подходящего простого числа.

5. Реализуйте работу оператора del (переопределите метод __delitem__) для удаления
элемента таблицы. Таблица должна автоматически менять размер, когда
загрузочный фактор становится меньше значения 0.2. Размер должен уменьшаться
примерно в два раза до ближайшего подходящего простого числа.

Все выполненные доработки должны быть протестированы.

In [1]:
class HashTable:
    def __init__(self):
        self.size = 11
        self.slots = [None] * self.size
        self.data = [None] * self.size
        
    def get_new_size(self, stop, step=1):
        start = self.size
        lst = [2, 3, 5]
        for i in range(start, stop, step):
            for j in lst:
                if i%j == 0:
                    break
            else:
                    lst.append(i)
        return lst[-1]

    def put(self, key, data):
        hashvalue = self.hashfunction(key, len(self.slots))

        if self.slots[hashvalue] == None:
            self.slots[hashvalue] = key
            self.data[hashvalue] = data
        else:
            if self.slots[hashvalue] == key:
                self.data[hashvalue] = data  # replace
            else:
                i = 1
                nextslot = self.rehash(hashvalue, len(self.slots), i)
                while self.slots[nextslot] != None and \
                        self.slots[nextslot] != key:
                    i += 1
                    nextslot = self.rehash(nextslot, len(self.slots), i)

                if self.slots[nextslot] == None:
                    self.slots[nextslot] = key
                    self.data[nextslot] = data
                else:
                    self.data[nextslot] = data  # replace
        _lambda_ = self.__len__() / self.size
        if _lambda_ > 0.7:
            last_size = self.size
            new_size = self.get_new_size(self.size * 2)
            delta_size = new_size - last_size
            self.size = new_size
            self.slots += [None] * delta_size
            self.data += [None] * delta_size
            

    def hashfunction(self, key, size):
        return key % size

    def rehash(self, oldhash, size, i):
        return (oldhash + i**2) % size

    def get(self, key):
        startslot = self.hashfunction(key, len(self.slots))

        data = None
        stop = False
        found = False
        position = startslot
        i = 1
        while self.slots[position] != None and \
                not found and not stop:
            if self.slots[position] == key:
                found = True
                data = self.data[position]
            else:
                position = self.rehash(position, len(self.slots), i)
                i += 1
                if position == startslot:
                    stop = True
        return data
    
    def delete(self, key):
        idx = -1
        for i in range(self.size):
            if self.slots[i] == key:
                idx = i
                break
        if idx >= 0:
            self.slots[idx] = None
            self.data[idx] = None
        
        _lambda_ = self.__len__() / self.size
        if _lambda_ < 0.2:
            slots = []
            data = []
            for i in range(self.size):
                if self.slots[i] != None:
                    slots.append(self.slots[i])
                    data.append(self.data[i])
            self.size = self.get_new_size(self.size // 2, -1)
            self.slots = [None] * self.size
            self.data = [None] * self.size
            for i in range(len(slots)):
                self.put(slots[i], data[i])

    def __getitem__(self, key):
        return self.get(key)

    def __setitem__(self, key, data):
        self.put(key, data)
        
    def __len__(self):
        count = 0
        for el in self.slots:
            if el != None:
                count += 1
        return count
    
    def __contains__(self, data):
        for el in self.data:
            if el == data:
                return True
        return False
    
    def __delitem__(self, key):
        self.delete(key)


H = HashTable()
H[54] = "cat"
H[26] = "dog"
H[93] = "lion"
H[17] = "tiger"
H[77] = "bird"
H[31] = "cow"
H[44] = "goat"
H[55] = "pig"
H[20] = "chicken"
print(H.slots)
print(H.data)

print(H[20])

print(H[17])
H[20] = 'duck'
print(H[20])
print(H[99])

print(len(H))

print("pig" in H)
print("amogus" in H)

print(H.slots)
print(H.data)
del H[20]
del H[44]
print(H.slots)
print(H.data)

[77, 44, 20, 55, 26, 93, 17, None, None, 31, 54, None, None, None, None, None, None, None, None]
['bird', 'goat', 'chicken', 'pig', 'dog', 'lion', 'tiger', None, None, 'cow', 'cat', None, None, None, None, None, None, None, None]
chicken
None
duck
None
9
True
False
[77, 44, 20, 55, 26, 93, 17, None, None, 31, 54, None, None, None, None, None, None, None, None]
['bird', 'goat', 'duck', 'pig', 'dog', 'lion', 'tiger', None, None, 'cow', 'cat', None, None, None, None, None, None, None, None]
[77, None, None, 55, 26, 93, 17, None, None, 31, 54, None, None, None, None, None, None, None, None]
['bird', None, None, 'pig', 'dog', 'lion', 'tiger', None, None, 'cow', 'cat', None, None, None, None, None, None, None, None]


### Задание №2 и №3
Возьмите реализацию класса HashTable из лекционных материалов и выполните
следующие доработки:

1. Переделайте существующие методы так, чтобы разрешение коллизий происходило
не при помощи концепции открытой адресации, а методом цепочек. Для этого в
каждом слоте храните связный список, реализованный классом UnorderedList из
лабораторной работы 3.

2. Реализуйте работу с функцией len (переопределите метод __len__).

3. Реализуйте работу оператора in (переопределите метод __contains__).

4. Переделайте метод put таким образом, чтобы таблица автоматически меняла размер,
когда загрузочный фактор становится больше значения 0.7. Размер должен
увеличиваться примерно в два раза до ближайшего подходящего простого числа.

5. Реализуйте работу оператора del (переопределите метод __delitem__) для удаления
элемента таблицы. Таблица должна автоматически менять размер, когдазагрузочный фактор становится меньше значения 0.2. Размер должен уменьшаться
примерно в два раза до ближайшего подходящего простого числа.

Все выполненные доработки должны быть протестированы.

Переделайте класс HashTable, чтобы в качестве ключей можно было использовать строки.

In [12]:
class Node:
    def __init__(self, initdata):
        self.data = initdata
        self.next = None

    def getData(self):
        return self.data

    def getNext(self):
        return self.next

    def setData(self, newdata):
        self.data = newdata

    def setNext(self, newnext):
        self.next = newnext


class UnorderedList:

    def __init__(self):
        self.head = None

    def isEmpty(self):
        return self.head == None

    def add(self, item):
        temp = Node(item)
        temp.setNext(self.head)
        self.head = temp

    def size(self):
        current = self.head
        count = 0
        while current != None:
            count = count + 1
            current = current.getNext()
        return count

    def search(self, item):
        current = self.head
        found = False
        while current != None and not found:
            if current.getData() == item:
                found = True
            else:
                current = current.getNext()
        return found

    def remove(self, item):
        current = self.head
        previous = None
        found = False
        while not found:
            if current.getData() == item:
                found = True
            else:
                previous = current
                current = current.getNext()

        if previous == None:
            self.head = current.getNext()
        else:
            previous.setNext(current.getNext())

    def append(self, item):
        current = self.head
        if current == None:
            self.add(item)
            return
        while current.getNext() != None:
            current = current.getNext()
        temp = Node(item)
        temp.setNext(None)
        current.setNext(temp)

    def index(self, item):
        current = self.head
        index = 0
        found = False
        while current != None and not found:
            if current.getData() == item:
                found = True
            else:
                index += 1
                current = current.getNext()
        if not found:
            index = None
        return index

    def insert(self, pos, item):
        if self.size() < pos or pos < 0:
            raise ("Index out of range")
        if pos == 0:
            self.add(item)
            return
        elif pos == self.size():
            self.append(item)
            return

        current = self.head
        for _ in range(pos-1):
            current = current.getNext()
        temp = Node(item)
        temp.setNext(current.getNext())
        current.setNext(temp)

    def pop(self, pos=None):
        if self.isEmpty():
            raise ("List is empty")
        current = self.head
        if pos == None:
            while current.getNext() != None:
                current = current.getNext()
        else:
            if self.size() - 1 < pos or pos < 0:
                raise ("Index out of range")
            for _ in range(pos):
                current = current.getNext()
        self.remove(current.getData())
        return current.getData()

    def __str__(self):
        current = self.head
        output = "["
        while current != None:
            output += str(current.getData()) + ", "
            current = current.getNext()
        if len(output) > 2:
            output = output[:-2] + "]"
        else:
            output += "]"
        return output

    def slice(self, start, stop):
        if start < 0 or stop < 0 or start > self.size() or stop > self.size():
            raise ("Index out of range")
        current = self.head
        rez = UnorderedList()
        for _ in range(start):
            current = current.getNext()
        for _ in range(start, stop):
            rez.append(current.getData())
            current = current.getNext()
        if rez.size() == 1:
            return rez.head.data
        return rez

    def __getitem__(self, item):
        if isinstance(item, slice):
            return self.slice(item.start, item.stop)

In [15]:
class HashTable:
    def __init__(self):
        self.size = 11
        self.slots = [None] * self.size
        self.data = [None] * self.size

    def get_new_size(self, stop, step=1):
        start = self.size
        lst = [2, 3, 5]
        for i in range(start, stop, step):
            for j in lst:
                if i%j == 0:
                    break
            else:
                    lst.append(i)
        return lst[-1]

    def put(self, key, data):
        hashvalue = self.hashfunction(key, len(self.slots))

        if self.slots[hashvalue] == None:
            self.slots[hashvalue] = UnorderedList()
            self.data[hashvalue] = UnorderedList()
            self.slots[hashvalue].append(key)
            self.data[hashvalue].append(data)
        else:
            if self.slots[hashvalue].search(key):
                idx = self.slots[hashvalue].index(key)
                self.data[hashvalue].pop(idx)
                self.data[hashvalue].insert(idx, data)
            else:
                self.slots[hashvalue].append(key)
                self.data[hashvalue].append(data)

        _lambda_ = self.__len__() / self.size
        if (_lambda_ > 0.7):
            last_size = self.size
            new_size = self.get_new_size(self.size * 2)
            delta_size = new_size - last_size
            self.size = new_size
            self.slots += [None] * delta_size
            self.data += [None] * delta_size

    def hashfunction(self, key, size):
        if isinstance(key, str):
            key = len(key)
        return key % size

    def rehash(self, oldhash, size):
        return (oldhash + 1) % size

    def get(self, key):
        startslot = self.hashfunction(key, len(self.slots))

        data = None
        stop = False
        found = False
        position = startslot
        while not found and not stop:
            if self.slots[position] != None:
                if self.slots[position].search(key):
                    found = True
                    idx = self.slots[position].index(key)
                    data = self.data[position].pop(idx)
                    self.data[position].insert(idx, data)

            position = self.rehash(position, len(self.slots))
            if position == startslot:
                stop = True

        _lambda_ = self.__len__() / self.size
        if (_lambda_ > 0.7):
            last_size = self.size
            new_size = self.get_new_size(self.size * 2)
            delta_size = new_size - last_size
            self.size = new_size
            self.slots += [None] * delta_size
            self.data += [None] * delta_size
        return data

    def __getitem__(self, key):
        return self.get(key)

    def __setitem__(self, key, data):
        self.put(key, data)

    def __len__(self):
        count = 0
        for el in self.slots:
            if el != None:
                count += 1
        return count

    def __contains__(self, data):
        for el in self.data:
            if el != None and el.search(data):
                return True
        return False

    def __delitem__(self, key):
        idxList = -1
        idxData = -1
        for i in range(self.size):
            if self.slots[i] != None and self.slots[i].search(key):
                idxList = i
                idxData = self.slots[i].index(key)
                break
        if idxData >= 0:
            self.slots[idxList].pop(idxData)
            self.data[idxList].pop(idxData)
            if self.slots[idxList].size() == 0:
                self.slots[idxList] = None
                self.data[idxList] = None
        
        _lambda_ = self.__len__() / self.size
        if _lambda_ < 0.2:
            slots = []
            data = []
            for i in range(self.size):
                if self.slots[i] != None:
                    for _ in range(self.slots[i].size()):
                        slots.append(self.slots[i].pop())
                        data.append(self.data[i].pop())
            self.size = self.get_new_size(self.size // 2, -1)
            self.slots = [None] * self.size
            self.data = [None] * self.size
            for i in range(len(slots)):
                self.put(slots[i], data[i])

In [18]:
H = HashTable()
H[54] = "cat"
H[26] = "dog"
H[93] = "lion"
H[17] = "tiger"
H[77] = "bird"
H[31] = "cow"
H[44] = "goat"
H[55] = "pig"
H[20] = "chicken"

for i in range(H.size):
    print(H.slots[i], end=" ")
    print(H.data[i])
    
print(len(H))

print("cow" in H)

print(H[20])

del H[20]
del H[31]
del H[26]
del H[93]

for i in range(H.size):
    print(H.slots[i], end=" ")
    print(H.data[i])
    
H["dog"] = "123"

for i in range(H.size):
    print(H.slots[i], end=" ")
    print(H.data[i])

[77, 44, 55] [bird, goat, pig]
None None
None None
None None
[26] [dog]
[93] [lion]
[17] [tiger]
None None
None None
[31, 20] [cow, chicken]
[54] [cat]
6
True
chicken
[77, 44, 55] [bird, goat, pig]
None None
None None
None None
None None
None None
[17] [tiger]
None None
None None
None None
[54] [cat]
[77, 44, 55] [bird, goat, pig]
None None
None None
[dog] [123]
None None
None None
[17] [tiger]
None None
None None
None None
[54] [cat]


### Задание №4
Дана строчка русского текста, состоящая из слов и пробелов. Словом считается
последовательность русских букв, слова разделены одним или большим числом пробелов.
Для каждого слова этого текста узнайте порядковый номер его вхождения в текст именно в
той форме, в которой указано слово. Для первого вхождения слова выведите «1», для
второго вхождения того же слова выведите «2» и так далее.
Для решения этой задачи используйте класс HashTable из задания № 3.

In [26]:
text = input().split()
H = HashTable()
for word in text:
    if H[word] == None:
        H[word] = 1
    else:
        H[word] = H[word] + 1
    print(H[word])

раз два три раз два пять
1
1
1
2
2
1


### Задание №5
Напишите программу, имитирующую процесс регистрации и авторизации. Для каждого
пользователя программа должна сохранять логин, хеш его пароля и «соль». Для хранения
данных можно использовать БД или файл.
Действия при сохранении пароля:

1. Сгенерируйте длинную случайную «соль» при помощи модуля secrets (secrets —
Generate secure random numbers for managing secrets — Python 3.10.0 documentation).
Длина «соли» должна быть такой же как и выходные данные используемой вами
хэш-функции. Например, если для хеширования вы используете SHA256, то на
выходе вы получите 256 бит (32 байта). В этом случае соль должна составлять не
менее 32 случайных байт.

2. Добавьте «соль» к паролю и хэшируйте его с помощью функции scrypt из модуля
hashlib (Хеширование паролей модулем hashlib в Python. (docs-python.ru)).

3. Сохраните логин, «соль» и получившейся хэш в БД или в файл.

Действия при проверке пароля:

1. Извлеките «соль» и хэш пользователя из БД или файла.

2. Добавьте «соль» к введенному паролю и хэшируйте его, используя ту же хэш-
функцию, что и в алгоритме сохранения пароля.3. Сравните получившейся хэш введенного пароля с хэшом из БД или файла. Если они
совпадают, то пароль правильный. В противном случае пароль был введен неверно.

In [29]:
import secrets
from hashlib import scrypt

In [44]:
def auth():
    login = input("Login: ")
    password = input("Password: ").encode()
    salt = None
    hash = None
    with open("5.txt", "r") as file:
        for line in file:
            row = line.split(";")
            if row[0] == login:
                salt = row[1].encode()
                hash = row[2]
    if salt == None or hash == None:
        print("Not found")
        return
    hashed_pass = scrypt(password=password, salt=salt, n=8, r=512, p=4, dklen=32).hex()
    if (hash == hashed_pass):
        print("Yes")
    else:
        print("No")

def reg():
    login = input("Login: ")
    password = input("Password: ").encode()
    salt = secrets.token_hex(32).encode()
    hashed_pass = scrypt(password=password, salt=salt, n=8, r=512, p=4, dklen=32).hex()
    with open("5.txt", "a") as file:
        file.write(login + ";" + salt.decode() + ";" + hashed_pass)

m = 1
while m != 0:
    print("1. authorization")
    print("2. registration")
    print("0. exit")
    m = int(input())
    if m == 1:
        auth()
    elif m == 2:
        reg()

1. authorization
2. registration
0. exit
2
Login: admin
Password: admin
1. authorization
2. registration
0. exit
1
Login: admin
Password: admin
Yes
1. authorization
2. registration
0. exit
1
Login: admin
Password: qwerty
No
1. authorization
2. registration
0. exit
0


### Задание №6
Напишите программу, которая принимает от пользователя путь до директории. Для всех
файлов из данной директории должен быть вычислен хеш. Программа должна выявить и
вывести на экран все дубликаты в этой директории (т.е. файлы, у которых одинаковый хеш).

In [64]:
import os
path = input("Путь:")
files = os.listdir(path) 
print(files)
H = HashTable()
for file in files:
    url = path + file
    hash = hashlib.sha256(open(url, 'rb').read()).hexdigest()
    if H[hash] == None:
        H[hash] = url
    else:
        print('Дубликаты:', H[hash], url)

Путь:./dir/
['1.txt', '11.txt', '2.txt', '22.txt', '3.txt']
Дубликаты: ./dir/1.txt ./dir/11.txt
Дубликаты: ./dir/2.txt ./dir/22.txt
